# PASTA Figure 3 Phenotype Map Generation

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tifffile
import pandas as pd
from pathlib import Path

from wrplot.segmentation import (
    create_rgb_annotation
)
from wrplot.utils import ax_plot_rgb_with_scalebar

## Phenotype Map Function

In [ ]:
def plot_phenotype(
    anno_path,
    tma_name,
    core_name,
    color_dict,
    cropping=False,
    crop_size=400,
    min_x=0,
    min_y=0,
    px = 1000,
    dpi = 300,
    saving=False,
    save_path="",
    format = "png"
):
    """
    Plot cell phenotype annotations as a colored segmentation mask.
    
    This function reads cell annotation data, loads a corresponding MESMER 
    segmentation mask, and creates an RGB visualization where each cell is 
    colored according to its phenotype classification.

    Parameters
    ----------
    anno_path : str
        Path to the CSV file containing cell annotations.
    tma_name : str
        Name of the tissue microarray (TMA). Here PASTA or CODEX.
    core_name : str
        Name of the specific core within the TMA.
    color_dict : dict
        Dictionary mapping phenotype classes to RGB color tuples.
    cropping : bool, default=False
        Whether to crop the image to a specific region.
    crop_size : int, default=400
        Size of the square crop in pixels.
    min_x : int, default=0
        X-coordinate of the top-left corner for cropping.
    min_y : int, default=0
        Y-coordinate of the top-left corner for cropping.
    px : int, default=1000
        Figure size in pixels when cropping is enabled.
    dpi : int, default=300
        Resolution in dots per inch.
    saving : bool, default=False
        Whether to save the figure to disk.
    save_path : str, default=""
        Path where the figure should be saved (without extension).
    format : str, default="png"
        Image format for saving (e.g., "png", "jpg", "svg").

    Returns
    -------
    None
        Displays the plot and optionally saves to disk.

    See Also
    --------
    create_rgb_annotation from wrplot.segmentation : Creates RGB annotation from segmentation mask.
    ax_plot_rgb_with_scalebar from wrplot.utils : Plots RGB image with scalebar.

    """



    ## Set up annotation dict
    anno_df = pd.read_csv(anno_path)

    tma_name_old = "A4" if tma_name == "PASTA" else "A3"

    subset = anno_df[
        (anno_df["TMA"] == tma_name_old) &
        (anno_df["core"] == core_name)
    ].copy()

    print(f"Rows for {tma_name}, {core_name}: {len(subset)}")

    subset["label"] = subset["cell_id"].str.lstrip("c").astype(int)

    ## Load segmentation mask
    segmentation_mask = tifffile.imread(f"./data/Fig3/01_MESMER_masks/Fig3_{tma_name}_{core_name}_MESMER_mask.tiff")

    unique_labels = np.unique(segmentation_mask)
    unique_labels = unique_labels[unique_labels != 0]
    
    ## Map segmentation to annotation dict
    subset["class"] = subset["annotation"].where(
        subset["annotation"].isin(color_dict.keys()),
        other="Others",
    )
    
    annotation_dict = dict(zip(subset["label"], subset["class"]))

    # Optional: restrict to only labels that exist in the current segmentation_mask
    unique_labels = np.unique(segmentation_mask)
    unique_labels = unique_labels[unique_labels != 0]

    annotation_dict = {lab: cls for lab, cls in annotation_dict.items() if lab in unique_labels}

    missing_in_ann = [lab for lab in unique_labels if lab not in annotation_dict]
    if missing_in_ann:
        print("WARNING: labels in mask but not in annotation table:", missing_in_ann[:20], "...")

    ## Creat RGB Image
    rgb_annotation = create_rgb_annotation(
        segmentation_mask,
        annotation_dict,
        color_dict,
        boundary=True,
    )

    ## Plot RGB Image
    H, W = segmentation_mask.shape

    if cropping:
        y_min = min_y
        y_max = min_y + crop_size
        x_min = min_x
        x_max = min_x + crop_size

        fig, ax = plt.subplots(1, 1,figsize=(px/dpi,px/dpi),dpi=dpi)

        ax_plot_rgb_with_scalebar(
            rgb_annotation[y_min:y_max, x_min:x_max],
            mpp=0.5,
            bar_width_um=0,
            bar_height_perc=0.01,
            text_to_bar_perc=0.01,
            text_size=20,
            plot_text=False,
            ax=ax,
        )

    else:
        fig, ax = plt.subplots(1, 1, figsize=(10, 10))

        ax_plot_rgb_with_scalebar(
            rgb_annotation,
            mpp=0.5,
            bar_width_um=0,
            bar_height_perc=0.01,
            text_to_bar_perc=0.01,
            text_size=20,
            ax=ax,
            plot_text = False
        )

    ax.axis("off")

    plt.subplots_adjust(left=0, right=1, top=1, bottom=0)

    if saving:
        save_path = save_path + "." + format
        Path(save_path).parent.mkdir(parents=True, exist_ok=True)

        plt.savefig(save_path, format=format, dpi=dpi, bbox_inches="tight", pad_inches=0)
        plt.close(fig)
    
    plt.show()

## Common variables

In [ ]:
color_dict_complete = {
    "CD4T": "#E41A1C",
    "CD8T": "#377EB8",
    "Treg": "#4DAF4A",
    "NK": "#984EA3",
    "DC": "#999999",
    "M1": "#FFFF33",
    "M2": "#A65628",
    "B": "#66C2A5",
    "Tumor": "#FC8D62"
}

annotation_file = "./data/Fig3/04_final_data/Fig3_final_annotation.csv"

## Figure 3B

### Hodgkin's

In [ ]:
#A4
plot_phenotype(
    anno_path=annotation_file,
    tma_name="PASTA",
    core_name="C-2",
    color_dict=color_dict_complete,
    cropping=False,
    saving=True,
    save_path="./output/Fig3/Fig3B_PhenotypeMaps/PASTA_C-2_map_full"
)

plot_phenotype(
    anno_path=annotation_file,
    tma_name="PASTA",
    core_name="C-2",
    color_dict=color_dict_complete,
    cropping=True,
    crop_size=400,
    min_x=1750,
    min_y=950,
    saving=True,
    save_path="./output/Fig3/Fig3B_PhenotypeMaps/PASTA_C-2_map_crop"
)

# A3
plot_phenotype(
    anno_path=annotation_file,
    tma_name="CODEX",
    core_name="C-2",
    color_dict=color_dict_complete,
    cropping=False,
    saving=True,
    save_path="./output/Fig3/Fig3B_PhenotypeMaps/CODEX_C-2_map_full"
)

plot_phenotype(
    anno_path=annotation_file,
    tma_name="CODEX",
    core_name="C-2",
    color_dict=color_dict_complete,
    cropping=True,
    crop_size=400,
    min_x=1750,
    min_y=950,
    saving=True,
    save_path="./output/Fig3/Fig3B_PhenotypeMaps/CODEX_C-2_map_crop"
)

### DLBCL

In [ ]:
#A4
plot_phenotype(
    anno_path=annotation_file,
    tma_name="PASTA",
    core_name="B-3",
    color_dict=color_dict_complete,
    cropping=False,
    saving=True,
    save_path="./output/Fig3/Fig3B_PhenotypeMaps/PASTA_B-3_map_full"
)

plot_phenotype(
    anno_path=annotation_file,
    tma_name="PASTA",
    core_name="B-3",
    color_dict=color_dict_complete,
    cropping=True,
    crop_size=400,
    min_x=3750,
    min_y=1700,
    saving=True,
    save_path="./output/Fig3/Fig3B_PhenotypeMaps/PASTA_B-3_map_crop"
)

#A3
plot_phenotype(
    anno_path=annotation_file,
    tma_name="CODEX",
    core_name="B-3",
    color_dict=color_dict_complete,
    cropping=False,
    saving=True,
    save_path="./output/Fig3/Fig3B_PhenotypeMaps/CODEX_B-3_map_full"
)

plot_phenotype(
    anno_path=annotation_file,
    tma_name="CODEX",
    core_name="B-3",
    color_dict=color_dict_complete,
    cropping=True,
    crop_size=400,
    min_x=3750,
    min_y=1700,
    saving=True,
    save_path="./output/Fig3/Fig3B_PhenotypeMaps/CODEX_B-3_map_crop"
)

## Figure S6A

Phenotype maps for Hodgkin's and DLBCL in Fig. S6A are identical to those used in Fig. 1B. The code below prints the additional Tonsil map needed to complete Fig. S6A.

In [ ]:
for TMA in ["CODEX", "PASTA"]:
    plot_phenotype(
        anno_path=annotation_file,
        tma_name=TMA,
        core_name="Tonsil",
        color_dict=color_dict_complete,
        cropping=True,
        crop_size=4500,
        min_x=8900,
        min_y=5800,
        saving=True,
        save_path=f"./output/SuppsFig3/FigS6/FigS6A_{TMA}_Tonsil_map_full",
        px=500,
        dpi=100
    )

    plot_phenotype(
        anno_path=annotation_file,
        tma_name=TMA,
        core_name="Tonsil",
        color_dict=color_dict_complete,
        cropping=True,
        crop_size=400,
        min_x=12000,
        min_y=6150,
        saving=True,
        save_path=f"./output/SuppsFig3/FigS6/FigS6A_{TMA}_Tonsil_map_crop",
        format="png",
        px=500,
        dpi=100
    )

## Figure S6B-C

In [ ]:
annotation = pd.read_csv("./data/Fig3/05_final_data/Fig3_final_annotation.csv")

annotation["core"].unique()

coords = {
    "CODEX.A-1": {"x": 2300, "y": 2400},
    "CODEX.A-3": {"x": 2900, "y": 2200},
    "CODEX.B-2": {"x": 1100, "y": 3250},
    "CODEX.B-3": {"x": 1800, "y": 2000},
    "CODEX.B-4": {"x": 3200, "y": 1600},
    "CODEX.C-1": {"x": 1200, "y": 1100},
    "CODEX.C-2": {"x": 1750, "y": 1050},
    "CODEX.C-3": {"x": 2750, "y": 1350},
    "CODEX.C-4": {"x": 2350, "y": 4150},
    "CODEX.D-2": {"x": 2400, "y": 3000},
    "CODEX.D-4": {"x": 1350, "y": 2250},
    "CODEX.E-1": {"x": 2250, "y": 3100},
    "CODEX.E-3": {"x": 2500, "y": 2700},
    "CODEX.Tonsil": {"x": 7700, "y": 6600},
    "PASTA.A-1": {"x": 2300, "y": 2400},
    "PASTA.A-3": {"x": 2950, "y": 2200},
    "PASTA.B-2": {"x": 1100, "y": 3200},
    "PASTA.B-3": {"x": 1750, "y": 2100},
    "PASTA.B-4": {"x": 3300, "y": 1600},
    "PASTA.C-1": {"x": 1200, "y": 1150},
    "PASTA.C-2": {"x": 1800, "y": 1050},
    "PASTA.C-3": {"x": 2750, "y": 1450},
    "PASTA.C-4": {"x": 2350, "y": 4100},
    "PASTA.D-2": {"x": 2400, "y": 3000},
    "PASTA.D-4": {"x": 1300, "y": 2150},
    "PASTA.E-1": {"x": 2200, "y": 3150},
    "PASTA.E-3": {"x": 2700, "y": 2800},
    "PASTA.Tonsil": {"x": 7700, "y": 6700},
}

In [ ]:
## Full core overview

for TMA in ["CODEX", "PASTA"]:
    for core in annotation["core"].unique():
        plot_phenotype(
            anno_path=annotation_file,
            tma_name=TMA,
            core_name=core,
            color_dict=color_dict_complete,
            saving=True,
            save_path=f"./output/SuppsFig3/FigS6/FigS6B_{TMA}_{core}_map_full",
            format="png",
            px=500,
            dpi=100
        )

        key = TMA + "." + core

        plot_phenotype(
            anno_path=annotation_file,
            tma_name=TMA,
            core_name=core,
            color_dict=color_dict_complete,
            cropping=True,
            crop_size=400,
            min_x=coords[key]["x"],
            min_y=coords[key]["y"],
            saving=True,
            save_path=f"./output/SuppsFig3/FigS6/FigS6C_{TMA}_{core}_map_crop",
            px=500,
            dpi=100
        )

## Figure S8C

In [ ]:
## Hodgkin's C-2

plot_phenotype(
    anno_path=annotation_file,
    tma_name="PASTA",
    core_name="C-2",
    color_dict=color_dict_complete,
    cropping=True,
    crop_size=150,
    min_x=1850,
    min_y=1600,
    saving=True,
    save_path="./output/SuppsFig3/FigS8C/Fig8C_C-2"
)

## DLBCL A-3

plot_phenotype(
    anno_path=annotation_file,
    tma_name="PASTA",
    core_name="A-3",
    color_dict=color_dict_complete,
    cropping=True,
    crop_size=150,
    min_x=1135,
    min_y=3550,
    saving=True,
    save_path="./output/SuppsFig3/FigS8C/Fig8C_A-3"
)

## Tonsil

plot_phenotype(
    anno_path=annotation_file,
    tma_name="PASTA",
    core_name="Tonsil",
    color_dict=color_dict_complete,
    cropping=True,
    crop_size=150,
    min_x=11150,
    min_y=7250,
    saving=True,
    save_path="./output/SuppsFig3/FigS8C/Fig8C_Tonsil"
)